<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/faster_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 参考にさせていただいたサイト　https://zenn.dev/tsuzukia/articles/1381e6c9a88577
!pip install yt-dlp faster_whisper

In [ ]:
import subprocess
from faster_whisper import WhisperModel
import time

YOUTUBE_ID = "hOYzB3Qa9DE" # Youtube ID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"

# Download audio from Youtube
def dl_yt(yt_url):
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True)

dl_yt(f"https://youtu.be/{YOUTUBE_ID}")

In [ ]:
def get_model():

  model_size = "large-v2"
  devices_compute = [["cuda", "float16"],
                     ["cuda", "int8_float16"],
                     ["cuda", "int8"],
                     ["cpu", "int8"],
  ]

  for i,j in devices_compute:
    yield i,j, WhisperModel(model_size, device=i, compute_type=j)

In [ ]:
%%time

for i,j, model in get_model():

  time_sta = time.perf_counter()

  segments, info = model.transcribe(AUDIO_FILE_NAME, beam_size=5)
  print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
  for segment in segments:
      print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

  time_end = time.perf_counter()
  tim = time_end- time_sta

  print(i,j)
  print(tim, " Sec")
  print("---------------------------")